In [ ]:
import os
import pandas as pd
import numpy as np
from reinforced_replenishment.nodes.visualisation.plots import ts_plot
from reinforced_replenishment.nodes.evaluation import error_metrics
from reinforced_replenishment.nodes.feature_engineering.abc_classification import abc_analyse
from reinforced_replenishment.nodes.feature_engineering.xyz_classification import xyz_analyse
from reinforced_replenishment.nodes.feature_engineering.demand_classification import classify_demand
from reinforced_replenishment.nodes.data_transformation.forecast_table_loader import ForecastTableLoader
from reinforced_replenishment.nodes.data_transformation.forecast_result_aggregator import ForecastResultAggregator
from reinforced_replenishment.nodes.data_transformation.forecast_long_to_wide import forecast_long_to_wide
from reinforced_replenishment.nodes.data_transformation.forecast_validator import forecast_validator
from reinforced_replenishment.nodes.data_transformation.pprint_forecast_configs import pprint_forecast_configs
from reinforced_replenishment.nodes.evaluation.compare_benchmarks import (
    compare_benchmarks, diebold_mariano_test
)
from reinforced_replenishment.nodes.feature_selection.feature_score import feature_score

In [ ]:
import plotly.express as px
import plotly.offline as pyo
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

In [ ]:
from sklearn.feature_selection import (
    f_classif,
    f_regression,
    mutual_info_classif,
    mutual_info_regression,
)

In [ ]:
# Do not show warnings
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
%reload_kedro

## Load pacemaker forecast
- add table demo-demand-forecast to POSTGRES_CONNECTION_STRING

In [ ]:
# Enter namespace
namespace = "demo"
# Get all created_at values
catalog.load(f"{namespace}.forecast_configs@postgresql").created_at.unique()

In [ ]:
# (Optionally) List of created_ats to be evaluated
# The following forecasts can be found in demo-demand-forecast
created_at_dict = {
    "2024-08-12 11:13:03.398283": "DefaultSagging",
    "2024-08-12 11:17:46.208923": "DefaultLocalExplainingPredictionModel",
    "2024-08-12 11:18:46.863222": "DefaultScalingPredictionModel",
}

In [ ]:
# Class to load forecasts, optionally filter on creation timestamps
fc_table = ForecastTableLoader(namespace, created_at=created_at_dict.keys())

In [ ]:
# Load foresight forecast
foresight_forecast = fc_table.foresight_forecast()

### Show forecast config 

In [ ]:
# Optionally use .style to print out entire cells
pprint_forecast_configs(
    fc_table.forecast_configs(),
    # show_only_diff=True,
    # crop_configs=True, # make config more clear
)  # .style

### Feature importance

In [ ]:
(
    fc_table.forecast_feature_importance()
    .replace({"created_at": created_at_dict})
    .groupby(["feature", "created_at"])
    .mean(numeric_only=True)
    .reset_index()
    .pivot(index="feature", columns="created_at", values="feature_importance")
    .assign(_mean=lambda x: x.mean(axis=1))
    .sort_values(by="_mean", ascending=False, axis=0)
    .round(3)
    .drop(columns=["_mean"])
).head(10)

## Load input table & customer forecast

In [ ]:
# Load truth
input_data = catalog.load("forecast.input_table@postgresql")

In [ ]:
customer_name = "Customer"  # Enter customer name here

dict_cols = {
    c.replace("group.", ""): c
    for c in foresight_forecast.columns
    if c.startswith("group.")
}
dict_cols.update({"date": "date", "value": "truth", "internal_forecast": customer_name})

df_input = (
    input_data[list(dict_cols.keys())]
    .rename(columns=dict_cols)
    .set_index(list(dict_cols.values()))
    .reset_index(["truth", customer_name])
    .fillna(0)
)

### Feature score via SelectKBest
Argument score_func can be f_classif, mutual_info_classif, f_regression, mutual_info_regression (default is f_classif) for classification/regression tasks, respectively.
- f_* : F-statistic, assumes linear relationships and normality
- mutual_info_*: Based on Mutual information theory. Does not only detect linear relationships. No assumptions about the underlying data distribution.
- Execution time:
    - f_classif < f_regression < mutual_info_regression < mutual_info_classif (takes very long)
- Correlation with forecast_feature_importance for demo-data:
    - f_classif (worst) < f_regression < mutual_info_classif < mutual_info_regression (best)

In [ ]:
# Feature Score via SelectKBest
feature_score(
    input_data.dropna(subset="value"), target="value", score_func=mutual_info_regression
)

## Merge pacemaker and customer forecast

In [ ]:
# Convert from long to wide format
# Optionally enter forecast prefix OR forecast_translation
forecast_table = forecast_long_to_wide(
    foresight_forecast,
    # forecast_prefix="Pacemaker ",
    forecast_translation=created_at_dict,
).merge(
    df_input.drop(columns=["truth"]), left_index=True, right_index=True, how="inner"
)

In [ ]:
total_history = forecast_table.drop(columns=["truth"]).merge(
    df_input.drop(columns=[customer_name]),
    left_index=True,
    right_index=True,
    how="outer",
)

In [ ]:
# Check if scope of the forecasts matches
forecast_validator(forecast_table)
# Optionally: drop NaNs to achieve equal scope
forecast_table = forecast_table.dropna()

## Plot timeseries

In [ ]:
ts_plot(
    df=total_history.reset_index(),
    x="date",
    y=forecast_table.columns,
    freq="ME",
    dropdown=["group.product", "group.city"],
    # color=['#118df2',"#EF553B", "#1e1e1e",'#8f8d89'], # Optionally add colors
)

# Forecast evaluation

## T-Test comparison 
Equal to the compare_benchmark route of the forecastpipeline API

In [ ]:
error = (
    ForecastResultAggregator(forecast_table)
    .groupby(["group.sku_id", "group.city", "year", "month"])
    .sum()
    .groupby(["group.sku_id", "group.city"])
    .rmse()
    .get_table()
).dropna()

benchmark = forecast_table.columns[0]
for col in [c for c in error.columns if c != benchmark]:
    print(compare_benchmarks(error, col, benchmark))

## Diebold Mariano
The Diebold-Mariano Test (DM Test) is a statistical test used to compare the forecast accuracy of two time series models. The test assesses whether the difference between the forecast errors of two models is significant, indicating that one model provides better forecasts than the other.

In [ ]:
columns = forecast_table.columns[0:2]
(
    forecast_table.groupby("max_date")
    .apply(lambda x: pd.Series(diebold_mariano_test(x, *columns, crit="MSE")))
    .rename(columns={0: "DM stat", 1: "p-Value"})
    .assign(
        significant=lambda x: x["p-Value"] < 0.05,
        better_FC=lambda x: np.where(x["DM stat"] < 0, columns[0], columns[1]),
    )
)

## Weighted monthly accuracy (1-WAPE) per product sorted by sales volume per product

In [ ]:
error = 1 - (
    ForecastResultAggregator(forecast_table)
    .groupby(["year", "month", "group.product", "group.sku_id"])
    .sum()
    .groupby(["group.product"])
    .wape()
    .get_table()
)

fig = px.bar(
    error,
    # color=error.index,
    range_y=[0, 1],
    width=100 + len(forecast_table.columns) * 160,
    height=400,
    text_auto=".1%",
    # color_discrete_sequence=['#118df2',"#EF553B"],
    barmode="group",
)
fig.show()

## Boxplot: Scaled SPEC metric per sku_id and city

In [ ]:
error = (
    ForecastResultAggregator(forecast_table)
    .groupby(["group.product", "group.city", "year", "month"])
    .sum()
    .groupby(["group.product", "group.city"])
    .sc_spec()
    .get_table()
    .round(2)
    .reset_index()
).melt(
    id_vars=["group.product", "group.city"],
    value_vars=[c for c in forecast_table.columns if c != "truth"],
    var_name="Forecast Methode",
    value_name="forecast",
)

fig = px.box(
    error,
    y="forecast",
    x="Forecast Methode",
    color="Forecast Methode",
    # range_y=[0,1],
    width=100 + len(forecast_table.columns) * 200,
    height=400,
)
fig.show()

## Histogram: Relative monthly deviation in % per sku_id and city

In [ ]:
error = (
    ForecastResultAggregator(forecast_table)
    .groupby(["group.sku_id", "group.city", "year", "month"])
    .sum()
    .get_table()
    .round(2)
    .reset_index()
).melt(
    id_vars=["group.sku_id", "group.city", "year", "month", "truth"],
    value_vars=[c for c in forecast_table.columns if c != "truth"],
    var_name="Forecast Methode",
    value_name="forecast",
)

error["ERROR"] = error["forecast"] - error["truth"]
error["ABS_ERROR"] = error["ERROR"].abs()
error["REL_ERROR"] = (error["ERROR"] / error["truth"]) * 100

In [ ]:
error_key = "REL_ERROR"
# Filter extreme values, only common
lower = error.replace([np.inf, -np.inf], np.nan).dropna()[error_key].quantile(0.001)
upper = error.replace([np.inf, -np.inf], np.nan).dropna()[error_key].quantile(0.999)

deviations_tmp = error[lambda x: x[error_key].gt(lower) & x[error_key].lt(upper)]
# Make sure that the forecasts have the same groups
deviations_tmp = deviations_tmp.groupby(
    ["year", "month", "group.sku_id", "group.city"]
).filter(lambda x: len(x) == deviations_tmp["Forecast Methode"].nunique())

In [ ]:
fig = px.histogram(
    deviations_tmp,
    x=error_key,
    color="Forecast Methode",
    marginal="box",  # or violin, rug
    barmode="group",
    nbins=100,
)

fig.update(
    layout_xaxis={
        "title": "Relativer Fehler (VORHERSAGE - IST)/IST [%]",
        "autorange": True,
        #'range':[0,1]
    },
    layout_yaxis={"title": "Anzahl"},
)
fig.show()
print(
    deviations_tmp.groupby(["Forecast Methode"])
    .agg(
        Median=(error_key, "median"),
        Mean=(error_key, "mean"),
        Anzahl=(error_key, "count"),
    )
    .round(2)
)

## Overall accuracy and over/underestimation per sku_id

In [ ]:
error = (
    ForecastResultAggregator(forecast_table)
    .groupby(["group.sku_id"])
    .sum()
    .get_table()
    .round(2)
    .reset_index()
).melt(
    id_vars=["group.sku_id", "truth"],
    value_vars=[c for c in forecast_table.columns if c != "truth"],
    var_name="Forecast Methode",
    value_name="forecast",
)

error["ERROR"] = error["forecast"] - error["truth"]
error["ABS_ERROR"] = error["ERROR"].abs()
error["REL_ERROR"] = (error["ERROR"] / error["truth"]) * 100
error["ABS_REL_ERROR"] = error["REL_ERROR"].abs()
error["Accuracy"] = 100 - error["ABS_REL_ERROR"]

In [ ]:
fig = px.scatter(
    error,
    x="REL_ERROR",
    y="Accuracy",
    size="truth",
    color="Forecast Methode",
    hover_name="group.sku_id",
    size_max=60,
    range_x=[-50, 50],
    range_y=[50, 120],
    title="Jährliche Prognosegenauigkeit und Über-/Unterschätzung",
    labels={
        "REL_ERROR": "Relativer Fehler (VORHERSAGE - IST)/IST [%]",
        "Accuracy": "Genauigkeit [%]",
    },
)
fig["layout"].pop("updatemenus")
fig.show()

## Demand classification
- Smooth: Regular, consistent demand with low variability
- Erratic: High variability in demand quantities but demand occurs frequently
- Intermittent: Low frequency of demand, with some time periods showing no demand at all.
- Lumpy: Both irregular demand occurrences and high variability in demand size.
- Sparse: (own definition): < 5 data points

In [ ]:
demand = classify_demand(
    total_history.reset_index(),
    "truth",
    "date",
    ["group.sku_id", "group.city"],
    "M",
    sparse_limit=5,
).set_index(["group.sku_id", "group.city"])[["classification"]]
# Add demand classification to full timeseries
forecast_table_class = pd.merge(
    forecast_table,
    demand,
    how="left",
    left_index=True,
    right_index=True,
).set_index(["classification"], append=True)
display(demand.classification.value_counts().to_frame())

In [ ]:
error = 1 - (
    ForecastResultAggregator(forecast_table_class)
    .groupby(["group.sku_id", "group.city", "classification", "date"])
    .sum()
    .groupby(["group.sku_id", "group.city", "classification"])
    .wape()
    .groupby(["classification"])
    .median()
    .get_table()
    .round(3)
)

fig = px.bar(
    error,
    width=100 + len(forecast_table.columns) * 160,
    height=400,
    text_auto=".1%",
    barmode="group",
)
fig.show()

## ABC-XYZ Analyse
#### ABC: Klassifizierung nach Absatzmenge
- A: Kombinationen mit einem Anteil von 80% am Gesamtabsatz
- B: Kombinationen mit einem Anteil von 15% am Gesamtabsatz
- C: Kombinationen mit einem Anteil von 5% am Gesamtabsatz
  
#### XYZ: Klassifizierung nach Vorhersagbarkeit (gemäß ARIMA)
- X: Relativ gute Vorhersagbarkeit (stabile Nachfrage, top 20%)
- Y: Mittlere Vorhersagbarkeit (moderate Schwankungen, next 30%)
- C: Schwer vorhersehbare Nachfrage (sporadischer Bedarf, übrige 50%)

In [ ]:
abc = abc_analyse(
    total_history.reset_index()[lambda x: (x.date >= "2021-07-01")]
    .groupby(["group.sku_id", "group.city"])
    .agg(truth=("truth", "sum")),
    "truth",
)
xyz = xyz_analyse(
    total_history.reset_index(),
    "truth",
    "date",
    ["group.sku_id", "group.city"],
    verbose=True,
)

In [ ]:
abc_xyz = pd.merge(
    xyz.set_index(["group.sku_id", "group.city"]),
    abc,
    left_index=True,
    right_index=True,
)[["XYZ", "ABC", "truth"]]

In [ ]:
# Add ABC, XYZ information to full timeseries
forecast_table_abc_xyz = pd.merge(
    forecast_table,
    abc_xyz.drop("truth", axis=1),
    how="left",
    left_index=True,
    right_index=True,
).set_index(["ABC", "XYZ"], append=True)

In [ ]:
print("Number of total items: ", len(abc_xyz))
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=["Number of items", "Sum of truth value", "Percentage"],
)

grouper = (
    abc_xyz.groupby(["ABC", "XYZ"])
    .agg(truth_count=("truth", "count"), truth_sum=("truth", "sum"))
    .assign(
        truth_all=lambda x: x.truth_sum.sum(),
        truth_pct=lambda x: x.truth_sum / x.truth_all,
    )
    .pivot_table(index="ABC", columns="XYZ", values=["truth_count", "truth_sum"])
)
# In numbers
fig.add_trace(
    px.imshow(grouper["truth_count"], text_auto=True, origin="lower").data[0],
    row=1,
    col=1,
)

# In tonnage
fig.add_trace(
    px.imshow(grouper["truth_sum"], text_auto="0.3s", origin="lower").data[0],
    row=1,
    col=2,
)

# In percentage
fig.add_trace(
    px.imshow(grouper["truth_pct"], text_auto=".1%", origin="lower").data[0],
    row=1,
    col=3,
)

fig.update_layout(title_x=0.2, width=900, height=400, showlegend=False, font_size=12)
fig.update_coloraxes(colorscale="rdbu_r")
fig.show()

## WAPE per cluster

In [ ]:
error = (
    ForecastResultAggregator(forecast_table_abc_xyz)
    .groupby(["year", "month", "group.sku_id", "ABC", "XYZ"])
    .sum()
    .groupby(["group.sku_id", "ABC", "XYZ"])
    .wape()
    .groupby(["ABC", "XYZ"])
    .median()
    .get_table()
    .round(2)
    .pivot_table(index="ABC", columns="XYZ")
)
pred_cols = error.columns.get_level_values(0).unique()
fig = make_subplots(rows=1, cols=len(pred_cols), subplot_titles=pred_cols)
for i, forecast in enumerate(pred_cols):
    fig.add_trace(px.imshow(error[forecast], text_auto=True).data[0], row=1, col=i + 1)

fig.update_layout(
    title="Wape",
    title_x=0.5,
    width=100 + len(pred_cols) * 220,
    height=350,
    showlegend=False,
    font_size=12,
)
fig.update_coloraxes(colorscale="rdbu")
fig.show()

## Correlation with external data

In [ ]:
target = "value"
date = "date"
groups = ["product", "city"]
externals = ["rain_amount", "temperature"]

df = input_data[[target, date, *groups, *externals]]

In [ ]:
# Add lags
for external in externals:
    for lag in range(1, 7):
        df[f"{external}_lag{lag}"] = df.groupby(groups)[external].shift(lag)
    for last in [3, 6]:
        df[f"{external}_last{last}"] = df.groupby(groups)[external].transform(
            lambda x: x.rolling(last, min_periods=1).mean()
        )
# Correlation between target and external columns
px.imshow(
    df.reindex(sorted(df.columns), axis=1)
    .drop(columns=groups)
    .corr()
    .loc[[target]]
    .drop(columns=[date, target])
    .round(2),
    height=250,
)

In [ ]:
def min_max_scaling(col):
    return (col - col.min()) / (col.max() - col.min())


df_scaled = df.copy()
df_scaled[[target, *externals]] = df_scaled[[target, *externals]].apply(min_max_scaling)

ts_plot(
    df=df_scaled,
    x="date",
    y=[target, *externals],
    freq="M",
    dropdown=groups,
)

## Export notebook

In [ ]:
os.system(
    f'jupyter nbconvert --output-dir="../data/08_reporting" --to html Evaluation_demo_data_retail.ipynb --no-input'
)